In [1]:
!pip install onnx -q
!pip install onnx-tf -q

import os
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import MobileNetV2
import tensorflow as tf
import torch
import onnx
from onnx_tf.backend import prepare

data_dir = os.path.abspath("../../data")
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), transforms.RandomErasing()])
trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
train_sub_len = int(len(trainset) * 0.001)
train_subset, val_subset = torch.utils.data.random_split(trainset, [train_sub_len, len(trainset) - train_sub_len])
trainloader = torch.utils.data.DataLoader(train_subset, batch_size=4, shuffle=True, num_workers=2)


model = MobileNetV2()

"""Fuse"""
# pair_of_modules_to_fuze = []
# for name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.Linear):
#         pair_of_modules_to_fuze.append([name.split('.')[-1]])
#     elif isinstance(layer, torch.nn.ReLU) and len(pair_of_modules_to_fuze) > 0:
#         pair_of_modules_to_fuze[-1].append(name.split('.')[-1])
# pair_of_modules_to_fuze = list(filter(lambda x: len(x) == 2, pair_of_modules_to_fuze))
# torch.quantization.fuse_modules(model.modules(), pair_of_modules_to_fuze, inplace=True)


"""Insert stubs"""
model = torch.nn.Sequential(torch.quantization.QuantStub(), 
                  model, 
                  torch.quantization.DeQuantStub())


"""Prepare"""
model.train()
model.qconfig = torch.quantization.get_default_qconfig("fbgemm")
torch.quantization.prepare_qat(model, inplace=True)


"""Training Loop"""
n_epochs = 3
opt = torch.optim.SGD(model.parameters(), lr=0.1)
loss_fn = torch.nn.CrossEntropyLoss()


for epoch in range(n_epochs):
    for inputs, labels in trainloader:
        opt.zero_grad()
        out = model(inputs)
        loss = loss_fn(out, labels)
        loss.backward()
        opt.step()


"""Convert"""
model.eval()
model_quantized = torch.quantization.convert(model, inplace=True)
torch.save(model_quantized, "../../results/MobileNetV2_TorchQATQuant.pt")


2023-03-07 10:44:43.120908: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 10:44:44.283148: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/slurm/slurm-19.05.5/lib:/usr/local/slurm/slurm-19.05.5/lib/slurm:/usr/local/lib:
2023-03-07 10:44:44.283250: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:

Files already downloaded and verified


/home/mnadar2s/anaconda3/envs/rnd/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [3]:
onnx_file = "../../results/mobilenet_torch_cifar.onnx"
tf_model_path = "../../results/mobilenet_torch_cifar.tf"

example_inputs = iter(trainloader)
images, lab = next(example_inputs)

# images = torch.rand(10, 3, 32, 32)

model = torch.load("../../results/MobileNetV2_TorchQATQuant.pt")
torch.onnx.export(model_quantized,               
                  images,                         
                  onnx_file,   
                  export_params=True,             
                  do_constant_folding=True,  
                  input_names = ['input'],   
                  output_names = ['output'], 
                  dynamic_axes={'input' : {0 : 'batch_size'},   
                                'output' : {0 : 'batch_size'}})



onnx_model =  onnx.load(onnx_file)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

NotImplementedError: Could not run 'aten::add.out' with arguments from the 'QuantizedCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::add.out' is only available for these backends: [CPU, CUDA, Meta, MkldnnCPU, SparseCPU, SparseCUDA, SparseCsrCPU, SparseCsrCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMeta, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradNestedTensor, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSn